In [1]:
import os
import torch
import torch.nn as nn
# import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [2]:
# 1) Setup the model
# checkpoint = "bigscience/bloomz-560m"
checkpoint = "bigscience/bigscience-small-testing"

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map='auto') # load_in_8bit=True,
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
# 2) Freeze the original weights
for param in model.parameters():
    param.requires_grad = False # freeze the model - train adapters later
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable() # reduce the number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

model.lm_head = CastOutputToFloat(model.lm_head)

In [4]:
# 3) Setting up the LoRA Adapters
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100*trainable_params/all_param}")
    
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, # attention heads
    lora_alpha=32, # alpha scaling TODO: Research what this is
    # target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8192 || all params: 16164736 || trainable%: 0.05067821707697546


In [5]:
# 4) Data
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")

Found cached dataset json (C:/Users/Alexis Strappazzon/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
data["train"]["quote"][:5]

['“Be yourself; everyone else is already taken.”',
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”",
 '“So many books, so little time.”',
 '“A room without books is like a body without a soul.”']

In [7]:
data["train"]["tags"][:5]

[['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst'],
 ['human-nature',
  'humor',
  'infinity',
  'philosophy',
  'science',
  'stupidity',
  'universe'],
 ['books', 'humor'],
 ['books', 'simile', 'soul']]

In [8]:
def merge_columns(example):
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example

data["train"] = data["train"].map(merge_columns)
data["train"]["prediction"][:5]

Loading cached processed dataset at C:\Users\Alexis Strappazzon\.cache\huggingface\datasets\Abirate___json\Abirate--english_quotes-6e72855d06356857\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-8a7463b65fe0bb13.arrow


["“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ->: ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ->: ['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
 "“So many books, so little time.” ->: ['books', 'humor']",
 "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"]

In [9]:
data['train'][0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 'prediction': "“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']"}

In [10]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

In [12]:
# 5) Training

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, # number of example per batch
        gradient_accumulation_steps=8, # number of batch to see before applying gradient update -> per_device_train_batch_size * gradient_accumulation_steps = number of example seen before an update
        warmup_steps=100, # starts with a very low lr and linearly goes up to the target lr every steps
        max_steps=1000, # number of steps after which the traning stops
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
trainer.train()

c:\Users\Alexis Strappazzon\Documents\code\python\HF_bloomz_test\.venv\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1000 [00:00<?, ?it/s]

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 12.4345, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 12.4354, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.01}
{'loss': 12.4311, 'learning_rate': 6e-06, 'epoch': 0.02}
{'loss': 12.4335, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.03}
{'loss': 12.4321, 'learning_rate': 1e-05, 'epoch': 0.03}
{'loss': 12.4321, 'learning_rate': 1.2e-05, 'epoch': 0.04}
{'loss': 12.435, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.04}
{'loss': 12.4335, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.05}
{'loss': 12.4359, 'learning_rate': 1.8e-05, 'epoch': 0.06}
{'loss': 12.4319, 'learning_rate': 2e-05, 'epoch': 0.06}
{'loss': 12.4338, 'learning_rate': 2.2000000000000003e-05, 'epoch': 0.07}
{'loss': 12.4331, 'learning_rate': 2.4e-05, 'epoch': 0.08}
{'loss': 12.4345, 'learning_rate': 2.6000000000000002e-05, 'epoch': 0.08}
{'loss': 12.4338, 'learning_rate': 2.8000000000000003e-05, 'epoch': 0.09}
{'loss': 12.4348, 'learning_rate': 3e-05, 'epoch': 0.1}
{'loss'

TrainOutput(global_step=1000, training_loss=12.425732020378113, metrics={'train_runtime': 301.4398, 'train_samples_per_second': 53.079, 'train_steps_per_second': 3.317, 'train_loss': 12.425732020378113, 'epoch': 6.38})

In [ ]:
trainer.save_model()

In [ ]:
# 6) Push model to hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

model.push_to_hub(user/repoid,
                  use_auth_token=True,
                  commit_message="basic training",
                  private=False)

In [ ]:
# Inference

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "username/repoid"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, device_map="auto") # load_in_8bits=True,
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [38]:
batch = tokenizer('“Training models with PEFT and LoRA is cool” ->: ', return_tensors='pt')
print(batch)
model.config.use_cache = True # silence the warnings. Please re-enable for inference!
model.eval()
with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=50)
    
print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))

{'input_ids': tensor([[ 86753,   7508,    386,  20038,   1002,    426, 108045,    530,   9810,
          14062,    632,  35847,    982,  11953,     29,    210]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


c:\Users\Alexis Strappazzon\Documents\code\python\HF_bloomz_test\.venv\lib\site-packages\transformers\generation\utils.py:1445: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 “Training models with PEFT and LoRA is cool” ->:  ['humor', 'modeling'], 'modeling', 'perception', 'strategies'], 'modeling'], 'modeling-institutes', 'perception'], 'perception', 'strateg
